In [ ]:
import base64
import json
from pygltflib import GLTF2, BufferFormat
import struct

from scipy.spatial.transform import Rotation as R
import numpy as np

DATA_URI_HEADER = "data:application/octet-stream;base64,"

In [ ]:
gltf = GLTF2.load('model_four.gltf') 

In [28]:
# construct a tree that specifies all the node's parents
# for each node, get the translation and rotation
def get_worldspace(nodes):
    tree = {}

    # populate tree with 66 nodes, each to an empty list
    for i in range(66):
        tree[i] = []
    for i, e in enumerate(gltf.nodes):
        if(i >= 66):
            break
        if(e.children is None):
            continue
        for child in e.children:
            tree[child].append(i)

    tree[65] = None
    # iterate through each node, and get the translation and rotation
    # transform the translation, rotation, and scale into the correct matrix
    # store this matrix within a list
    list = []

    for key, val in tree.items():
        node = gltf.nodes[key]
        translation = node.translation
        rotation = R.from_quat(node.rotation)
        scale = node.scale
        temp = np.zeros((4,4))
        temp[0:3, 0:3] = rotation.as_matrix()
        temp[0:3, 3] = translation
        temp[3,3] = 1
        list.append(temp)
    # iterate through each node, recursively multiply the matrix by the parent's matrix
    # transform the matrix back into the world space translation vector and rotation quaternion
    # store this translation and rotation within a list
    translations = []
    rotations = []

    for key, val in tree.items():
        node = gltf.nodes[key]
        itr = val
        # print(itr)
        while itr is not None:
            itr = itr[0]
            temp = list[itr]
            temp = np.matmul(temp, list[key])
            list[key] = temp
            itr = tree[itr]
        translation = list[key][0:3, 3]
        rotation = R.from_matrix(list[key][0:3, 0:3]).as_quat()
        # print(translation, rotation)
        translations.append(translation)
        rotations.append(rotation)
    return translations, rotations
    

In [29]:
position_list           = []
normal_list             = []
texcoord_list           = []
triangle_indicies_list  = []
bone_weight_list        = []
bone_indicies_list      = []
# bone_rest_pos           = []
# bone_rest_rot           = []
for mesh in gltf.meshes:
    for primitive in mesh.primitives:
        position_list.append(primitive.attributes.POSITION)
        normal_list.append(primitive.attributes.NORMAL)
        texcoord_list.append(primitive.attributes.TEXCOORD_0)
        triangle_indicies_list.append(primitive.indices)
        
        bone_weight_list.append(primitive.attributes.WEIGHTS_0)
        bone_indicies_list.append(primitive.attributes.JOINTS_0)

all_lists = [position_list, normal_list, texcoord_list, triangle_indicies_list, bone_weight_list, bone_indicies_list]
all_lists


[[0, 7, 13, 19],
 [1, 8, 14, 20],
 [2, 9, 15, 21],
 [5, 12, 18, 25],
 [4, 11, 17, 24],
 [3, 10, 16, 23]]

In [30]:
order = [65, 64, 63, 62, 61, 17, 16, 38, 37, 35, 33, 20, 19, 18, 23, 22, 21, 26, 25, 24, 29, 28, 27, 32, 31, 30, 34, 36, 59, 58, 56, 54, 41, 40, 39, 44, 43, 42,47, 46, 45, 50, 49, 48, 53, 52, 51, 55, 57, 60, 14, 15, 6, 2, 1, 0, 5, 4, 13, 9, 8, 7, 10, 11, 12]
print(len(order))
for i in order:
    print('\'' + gltf.nodes[i].name + '\' : '+ str(i) + ',')

65
'Bip002' : 65,
'Bip002_Pelvis' : 64,
'Bip002_Spine' : 63,
'Bip002_Spine1' : 62,
'Bip002_Spine2' : 61,
'Bip002_Neck' : 17,
'Bip002_Head' : 16,
'Bip002_L_Clavicle' : 38,
'Bip002_L_UpperArm' : 37,
'Bip002_L_Forearm' : 35,
'Bip002_L_Hand' : 33,
'Bip002_L_Finger0' : 20,
'Bip002_L_Finger01' : 19,
'Bip002_L_Finger02' : 18,
'Bip002_L_Finger1' : 23,
'Bip002_L_Finger11' : 22,
'Bip002_L_Finger12' : 21,
'Bip002_L_Finger2' : 26,
'Bip002_L_Finger21' : 25,
'Bip002_L_Finger22' : 24,
'Bip002_L_Finger3' : 29,
'Bip002_L_Finger31' : 28,
'Bip002_L_Finger32' : 27,
'Bip002_L_Finger4' : 32,
'Bip002_L_Finger41' : 31,
'Bip002_L_Finger42' : 30,
'Bone_L_ForeArm_Twist' : 34,
'Bone_L_UpperArm_Twist' : 36,
'Bip002_R_Clavicle' : 59,
'Bip002_R_UpperArm' : 58,
'Bip002_R_Forearm' : 56,
'Bip002_R_Hand' : 54,
'Bip002_R_Finger0' : 41,
'Bip002_R_Finger01' : 40,
'Bip002_R_Finger02' : 39,
'Bip002_R_Finger1' : 44,
'Bip002_R_Finger11' : 43,
'Bip002_R_Finger12' : 42,
'Bip002_R_Finger2' : 47,
'Bip002_R_Finger21' : 46,
'Bip002_

In [31]:
for i in range(66):
    print( '\'' + gltf.nodes[i].name + '\' : '+ str(i) + ',')

'Bip002_L_Toe0' : 0,
'Bip002_L_Foot' : 1,
'Bip002_L_Calf' : 2,
'Bone011' : 3,
'Bone010' : 4,
'Bone009' : 5,
'Bip002_L_Thigh' : 6,
'Bip002_R_Toe0' : 7,
'Bip002_R_Foot' : 8,
'Bip002_R_Calf' : 9,
'Bone008' : 10,
'Bone006' : 11,
'Bone013' : 12,
'Bip002_R_Thigh' : 13,
'Bone007' : 14,
'Bone014' : 15,
'Bip002_Head' : 16,
'Bip002_Neck' : 17,
'Bip002_L_Finger02' : 18,
'Bip002_L_Finger01' : 19,
'Bip002_L_Finger0' : 20,
'Bip002_L_Finger12' : 21,
'Bip002_L_Finger11' : 22,
'Bip002_L_Finger1' : 23,
'Bip002_L_Finger22' : 24,
'Bip002_L_Finger21' : 25,
'Bip002_L_Finger2' : 26,
'Bip002_L_Finger32' : 27,
'Bip002_L_Finger31' : 28,
'Bip002_L_Finger3' : 29,
'Bip002_L_Finger42' : 30,
'Bip002_L_Finger41' : 31,
'Bip002_L_Finger4' : 32,
'Bip002_L_Hand' : 33,
'Bone_L_ForeArm_Twist' : 34,
'Bip002_L_Forearm' : 35,
'Bone_L_UpperArm_Twist' : 36,
'Bip002_L_UpperArm' : 37,
'Bip002_L_Clavicle' : 38,
'Bip002_R_Finger02' : 39,
'Bip002_R_Finger01' : 40,
'Bip002_R_Finger0' : 41,
'Bip002_R_Finger12' : 42,
'Bip002_R_Finger11

In [32]:
with open("character_sofunny_model_four.bin", "wb") as f:
    written_bytes = 0
    ROWS = 4
    uri = gltf.buffers[0].uri
    all_data = uri.split(DATA_URI_HEADER)
    data = uri.split(DATA_URI_HEADER)[1]
    data = base64.b64decode(bytes(data,"utf8"))
    for attribute in all_lists:
        # get total count of current attribute
        # get data locations for current attribute
        total_count_of_attribute = 0
        to_write = []
        for accessor_index in attribute:
            accessor = gltf.accessors[accessor_index]
            total_count_of_attribute += accessor.count
            buffer_view = gltf.bufferViews[accessor.bufferView]
            byte_offset = buffer_view.byteOffset
            byte_length = buffer_view.byteLength
            to_write.append((byte_offset, byte_length))

        # write data to file
        print(to_write)

        # write total count of current attribute
        f.write(total_count_of_attribute.to_bytes(4, byteorder='little', signed=False))
        if(attribute[0] == 4 or attribute[0] == 3):
            f.write(ROWS.to_bytes(4, byteorder='little', signed=False))
        # write data locations for current attribute
        for byte_offset, byte_length in to_write:
            f.write(data[byte_offset:byte_offset+byte_length])
            written_bytes += byte_length
        print(written_bytes)

        
    tc = len(gltf.nodes)-5
    print(tc)
    translations, rotations = get_worldspace(gltf.nodes)
    f.write(tc.to_bytes(4, byteorder='little', signed=False))
    order = [65, 64, 63, 62, 61, 17, 16, 38, 37, 35, 33, 20, 19, 18, 23, 22, 21, 26, 25, 24, 29, 28, 27, 32, 31, 30, 34, 36, 59, 58, 56, 54, 41, 40, 39, 44, 43, 42,47, 46, 45, 50, 49, 48, 53, 52, 51, 55, 57, 60, 14, 15, 6, 2, 1, 0, 5, 4, 13, 9, 8, 7, 10, 11, 12]
    # print("hi\t", str(len(order)))
    f.write(struct.pack('<fff', 0.0, 0.0, 0.0))
    # for translation in translations:
    for x in order:
        translation = translations[x]
        for i in range(3):
            f.write(struct.pack('<f', translation[i]/100))

    f.write(tc.to_bytes(4, byteorder='little', signed=False))
    f.write(struct.pack('<ffff', 0.0, 0.0, 0.0, 1.0))
    # for rotation in rotations:
    for x in order:
        rotation = rotations[x]
        for i in range(4):
            f.write(struct.pack('<f', rotation[i]))
    
    

[(0, 108192), (10176932, 106068), (20101436, 48780), (20354368, 70128)]
333168
[(108192, 108192), (10283000, 106068), (20150216, 48780), (20424496, 70128)]
666336
[(216384, 72128), (10389068, 70712), (20198996, 32520), (20494624, 46752)]
888448
[(468832, 84582), (10636560, 73158), (20312816, 41550), (20705008, 50130)]
1137868
[(324576, 144256), (10495136, 141424), (20247776, 65040), (20611504, 93504)]
1582092
[(288512, 36064), (10459780, 35356), (20231516, 16260), (20588128, 23376)]
1693148
66


In [ ]:
### deprecated, please do not use.
x = 0
with open("character_sofunny_model_four.bin", "wb") as f: 
    uri = gltf.buffers[0].uri
    all_data = uri.split(DATA_URI_HEADER)
    data = uri.split(DATA_URI_HEADER)[1]
    data = base64.b64decode(bytes(data,"utf8"))
    total_list = []
    for attributes in all_lists:
        total_counter = 0
        
        #get data from uri

        #start and ending bytes of each list
        temp_list = []

        for temp_index in attributes:
            print("temp index:\t", str(temp_index))
            accessor_index = gltf.accessors[temp_index].bufferView
            print("accessor_index:\t", str(accessor_index))
            total_counter += gltf.accessors[temp_index].count
            print(gltf.accessors[temp_index].count, end=" ")
            print()
            byteOS = gltf.bufferViews[accessor_index].byteOffset
            bytelength = gltf.bufferViews[accessor_index].byteLength
            temp_list.append((byteOS, byteOS + bytelength, accessor_index))
        # print(total_counter)
        f.write(total_counter.to_bytes(4, byteorder='little', signed=False))
        if(x == 4 or x == 5):
            temp = 4
            f.write(temp.to_bytes(4, byteorder='little', signed=False))
        total_list.append(total_counter)
        for start, end, accessor_index in temp_list:
            x += 1
            print(str(x) +" " + gltf.accessors[temp_index].type)
            print('\t', end="")
            print(start, end)
            print('\t', end="")
            print(gltf.bufferViews[accessor_index].byteLength, end-start)
            print('\t', end="")
            f.write(data[start:end])

    tc = len(gltf.nodes)
    world_space = get_worldspace(gltf.nodes)
    position, rotation = world_space
    f.write(tc.to_bytes(4, byteorder='little', signed=False))
    
    f.write(tc.to_bytes(4, byteorder='little', signed=False))

            

    # look through each list, add total count of the 

In [ ]:
gltf.nodes[65]

In [ ]:
total_list

In [ ]:
print(struct.unpack('H', data[20611504:20611504+2]))

In [ ]:
x = 468832    
x = 10636624
x = 20312880
x = 20705072
x = x + 2 * 10000
for i in range(100):
    y = struct.unpack("H", data[x:x+2])
    x += 2
    print(y)

In [ ]:
with open('character.bin', 'rb') as f:
    print(int.from_bytes(f.read(4), byteorder='little', signed=False))
    


In [ ]:
# test

with open("character.bin", "rb") as f:
    types = ['vec3', 'vec3', 'vec2', 'unsigned short', '2dfloat', '2dus', 'vec3', 'quat']
    for type in types:
        count = int.from_bytes(f.read(4), byteorder='little', signed=False)
        print(count)
        match type:
            case 'vec3':
                for i in range(count):
                    x = struct.unpack("HHH", f.read(6))
                    print(x)
                    break
        

In [ ]:

# transfrom the translations of the character from local space to world space
# transfrom the rotations of the character from local space to world space



In [12]:
for i in range(66):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


In [23]:
names = [node.name for node in gltf.nodes]
names2 = [gltf.nodes[i].name for i in order]
print(names)
print(names2)

['Bip002_L_Toe0', 'Bip002_L_Foot', 'Bip002_L_Calf', 'Bone011', 'Bone010', 'Bone009', 'Bip002_L_Thigh', 'Bip002_R_Toe0', 'Bip002_R_Foot', 'Bip002_R_Calf', 'Bone008', 'Bone006', 'Bone013', 'Bip002_R_Thigh', 'Bone007', 'Bone014', 'Bip002_Head', 'Bip002_Neck', 'Bip002_L_Finger02', 'Bip002_L_Finger01', 'Bip002_L_Finger0', 'Bip002_L_Finger12', 'Bip002_L_Finger11', 'Bip002_L_Finger1', 'Bip002_L_Finger22', 'Bip002_L_Finger21', 'Bip002_L_Finger2', 'Bip002_L_Finger32', 'Bip002_L_Finger31', 'Bip002_L_Finger3', 'Bip002_L_Finger42', 'Bip002_L_Finger41', 'Bip002_L_Finger4', 'Bip002_L_Hand', 'Bone_L_ForeArm_Twist', 'Bip002_L_Forearm', 'Bone_L_UpperArm_Twist', 'Bip002_L_UpperArm', 'Bip002_L_Clavicle', 'Bip002_R_Finger02', 'Bip002_R_Finger01', 'Bip002_R_Finger0', 'Bip002_R_Finger12', 'Bip002_R_Finger11', 'Bip002_R_Finger1', 'Bip002_R_Finger22', 'Bip002_R_Finger21', 'Bip002_R_Finger2', 'Bip002_R_Finger32', 'Bip002_R_Finger31', 'Bip002_R_Finger3', 'Bip002_R_Finger42', 'Bip002_R_Finger41', 'Bip002_R_Finge

In [34]:
for i in range(65):
    print(gltf.nodes[order[i]].name, i)

Bip002 0
Bip002_Pelvis 1
Bip002_Spine 2
Bip002_Spine1 3
Bip002_Spine2 4
Bip002_Neck 5
Bip002_Head 6
Bip002_L_Clavicle 7
Bip002_L_UpperArm 8
Bip002_L_Forearm 9
Bip002_L_Hand 10
Bip002_L_Finger0 11
Bip002_L_Finger01 12
Bip002_L_Finger02 13
Bip002_L_Finger1 14
Bip002_L_Finger11 15
Bip002_L_Finger12 16
Bip002_L_Finger2 17
Bip002_L_Finger21 18
Bip002_L_Finger22 19
Bip002_L_Finger3 20
Bip002_L_Finger31 21
Bip002_L_Finger32 22
Bip002_L_Finger4 23
Bip002_L_Finger41 24
Bip002_L_Finger42 25
Bone_L_ForeArm_Twist 26
Bone_L_UpperArm_Twist 27
Bip002_R_Clavicle 28
Bip002_R_UpperArm 29
Bip002_R_Forearm 30
Bip002_R_Hand 31
Bip002_R_Finger0 32
Bip002_R_Finger01 33
Bip002_R_Finger02 34
Bip002_R_Finger1 35
Bip002_R_Finger11 36
Bip002_R_Finger12 37
Bip002_R_Finger2 38
Bip002_R_Finger21 39
Bip002_R_Finger22 40
Bip002_R_Finger3 41
Bip002_R_Finger31 42
Bip002_R_Finger32 43
Bip002_R_Finger4 44
Bip002_R_Finger41 45
Bip002_R_Finger42 46
Bone_R_ForeArm_Twist 47
Bone_R_UpperArm_Twist 48
Bone004 49
Bone007 50
Bone0

In [25]:
for i in order:
    print(gltf.nodes[i].name, i)

Bip002 65
Bip002_Pelvis 64
Bip002_Spine 63
Bip002_Spine1 62
Bip002_Spine2 61
Bip002_Neck 17
Bip002_Head 16
Bip002_L_Clavicle 38
Bip002_L_UpperArm 37
Bip002_L_Forearm 35
Bip002_L_Hand 33
Bip002_L_Finger0 20
Bip002_L_Finger01 19
Bip002_L_Finger02 18
Bip002_L_Finger1 23
Bip002_L_Finger11 22
Bip002_L_Finger12 21
Bip002_L_Finger2 26
Bip002_L_Finger21 25
Bip002_L_Finger22 24
Bip002_L_Finger3 29
Bip002_L_Finger31 28
Bip002_L_Finger32 27
Bip002_L_Finger4 32
Bip002_L_Finger41 31
Bip002_L_Finger42 30
Bone_L_ForeArm_Twist 34
Bone_L_UpperArm_Twist 36
Bip002_R_Clavicle 59
Bip002_R_UpperArm 58
Bip002_R_Forearm 56
Bip002_R_Hand 54
Bip002_R_Finger0 41
Bip002_R_Finger01 40
Bip002_R_Finger02 39
Bip002_R_Finger1 44
Bip002_R_Finger11 43
Bip002_R_Finger12 42
Bip002_R_Finger2 47
Bip002_R_Finger21 46
Bip002_R_Finger22 45
Bip002_R_Finger3 50
Bip002_R_Finger31 49
Bip002_R_Finger32 48
Bip002_R_Finger4 53
Bip002_R_Finger41 52
Bip002_R_Finger42 51
Bone_R_ForeArm_Twist 55
Bone_R_UpperArm_Twist 57
Bone004 60
Bone00

In [35]:
for i in range(66):
    print(gltf.nodes[i].name, i)

Bip002_L_Toe0 0
Bip002_L_Foot 1
Bip002_L_Calf 2
Bone011 3
Bone010 4
Bone009 5
Bip002_L_Thigh 6
Bip002_R_Toe0 7
Bip002_R_Foot 8
Bip002_R_Calf 9
Bone008 10
Bone006 11
Bone013 12
Bip002_R_Thigh 13
Bone007 14
Bone014 15
Bip002_Head 16
Bip002_Neck 17
Bip002_L_Finger02 18
Bip002_L_Finger01 19
Bip002_L_Finger0 20
Bip002_L_Finger12 21
Bip002_L_Finger11 22
Bip002_L_Finger1 23
Bip002_L_Finger22 24
Bip002_L_Finger21 25
Bip002_L_Finger2 26
Bip002_L_Finger32 27
Bip002_L_Finger31 28
Bip002_L_Finger3 29
Bip002_L_Finger42 30
Bip002_L_Finger41 31
Bip002_L_Finger4 32
Bip002_L_Hand 33
Bone_L_ForeArm_Twist 34
Bip002_L_Forearm 35
Bone_L_UpperArm_Twist 36
Bip002_L_UpperArm 37
Bip002_L_Clavicle 38
Bip002_R_Finger02 39
Bip002_R_Finger01 40
Bip002_R_Finger0 41
Bip002_R_Finger12 42
Bip002_R_Finger11 43
Bip002_R_Finger1 44
Bip002_R_Finger22 45
Bip002_R_Finger21 46
Bip002_R_Finger2 47
Bip002_R_Finger32 48
Bip002_R_Finger31 49
Bip002_R_Finger3 50
Bip002_R_Finger42 51
Bip002_R_Finger41 52
Bip002_R_Finger4 53
Bip002

In [71]:
gltf.nodes[16].name

'Bip002_Head'

In [42]:
gltf.meshes[3].primitives[0].attributes.JOINTS_0

24

In [64]:
print(gltf.accessors[16].bufferView)
print(gltf.accessors[23].bufferView)
print(gltf.accessors[7])

18
25
Accessor(extensions={}, extras={}, bufferView=8, byteOffset=0, componentType=5126, normalized=False, count=8839, type='VEC3', sparse=None, max=[0.31501373648643494, 1.498440146446228, 0.23501354455947876], min=[-0.2885441482067108, 0.4760053753852844, -0.26382237672805786], name=None)


In [68]:
BOS = gltf.bufferViews[18].byteOffset
print(BOS)
x = data[BOS+1000:BOS+1100]
for i in range(50):
    print(struct.unpack(">B", x[i:i+1]))

20231516
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)


In [67]:
# head 
gltf.meshes[3]
BOS = gltf.bufferViews[25].byteOffset
x = data[BOS+1000:BOS+1100]
print(BOS)
x[:1]
for i in range(50):
    print(struct.unpack(">B", x[i:i+1]))

20588128
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)
(0,)
(0,)
(22,)
(0,)


In [91]:
gltf.skins[0].joints[22]

16

In [96]:
bone_i_convert = {}
for i in range(66):
    joint = gltf.nodes[gltf.skins[0].joints[i]]
    bone_i_convert[joint.name] = gltf.skins[0].joints[i]

In [97]:
bone_i_convert

{'Bip002': 65,
 'Bip002_Pelvis': 64,
 'Bip002_L_Thigh': 6,
 'Bip002_L_Calf': 2,
 'Bip002_L_Foot': 1,
 'Bip002_L_Toe0': 0,
 'Bone009': 5,
 'Bone010': 4,
 'Bone011': 3,
 'Bip002_R_Thigh': 13,
 'Bip002_R_Calf': 9,
 'Bip002_R_Foot': 8,
 'Bip002_R_Toe0': 7,
 'Bone008': 10,
 'Bone006': 11,
 'Bone013': 12,
 'Bip002_Spine': 63,
 'Bip002_Spine1': 62,
 'Bone007': 14,
 'Bone014': 15,
 'Bip002_Spine2': 61,
 'Bip002_Neck': 17,
 'Bip002_Head': 16,
 'Bip002_L_Clavicle': 38,
 'Bip002_L_UpperArm': 37,
 'Bip002_L_Forearm': 35,
 'Bip002_L_Hand': 33,
 'Bip002_L_Finger0': 20,
 'Bip002_L_Finger01': 19,
 'Bip002_L_Finger02': 18,
 'Bip002_L_Finger1': 23,
 'Bip002_L_Finger11': 22,
 'Bip002_L_Finger12': 21,
 'Bip002_L_Finger2': 26,
 'Bip002_L_Finger21': 25,
 'Bip002_L_Finger22': 24,
 'Bip002_L_Finger3': 29,
 'Bip002_L_Finger31': 28,
 'Bip002_L_Finger32': 27,
 'Bip002_L_Finger4': 32,
 'Bip002_L_Finger41': 31,
 'Bip002_L_Finger42': 30,
 'Bone_L_ForeArm_Twist': 34,
 'Bone_L_UpperArm_Twist': 36,
 'Bip002_R_Clavicle